In [75]:
import os
from elasticsearch import Elasticsearch 
import random
from collections import OrderedDict
from nltk.stem import PorterStemmer
import numpy
import re

In [35]:
import elasticsearch_dsl

In [8]:
es=Elasticsearch()

In [68]:
index_name='trec_spam'

In [69]:
query={
    'query': 
    {'query_string': 
     {"default_field": "split",
      "query": "training"
     }
    },"size": 75419}

In [83]:
features = dict()
word_dict = dict()
res=es.search(index=index_name,doc_type='document',body=query)
ids = [d['_id'] for d in res['hits']['hits']]
for ID in ids:
    body=es.get(index=index_name, doc_type='document', id=ID)['_source']['body']
    text=re.sub(" \d+", " ", body)
    text=text.replace('   ',' ')
    text=text.replace('  ',' ')
    terms = text.split()
    for term in terms:
        features[term]=term
count=0
for term in features:
    count+=1
    word_dict[term]=count
        
    


In [93]:
sparse_matrix_train=open('../sparse_matrix_train.txt','a')
for ID in ids:
    term_freq=dict()
    label=es.get(index=index_name, doc_type='document',id=ID)['_source']['label']
    if label=='spam':
        ID_label='1'
    else:
        ID_label='0'
    
    body=es.get(index=index_name, doc_type='document',id=ID)['_source']['body']
    text=re.sub(" \d+", " ", body)
    text=text.replace('   ',' ')
    text=text.replace('  ',' ')
    
    texts=text.split()
    word_seq=dict()
    for text in texts:
        word_seq[str(text)]=word_dict[str(text)]
        try:
            term_freq[str(text)]+=1
        except:
            term_freq[str(text)]=1
    
    final=list()
    check_word=dict()
    
    for word in sorted(word_seq, key=word_seq.get):
        if word in check_word:
            continue
        else:
            check_word[word]=word
            final.append(str(word_dict[word]) + ":" + str(term_freq[word]))
            
            
    final_list=' '.join(final)
    sparse_matrix_train.write(ID_label+" "+final_list+"\n")
sparse_matrix_train.close()      


In [99]:
sparse_matrix_test=open('../sparse_matrix_test.txt','a')
test_query={
    'query': 
    {'query_string': 
     {"default_field": "split",
      "query": "testing"
     }
    },"size": 75419}
test_res=es.search(index=index_name,doc_type='document',body=test_query)
test_ids = [d['_id'] for d in test_res['hits']['hits']]

for ID in test_ids:
    term_freq=dict()
    label=es.get(index=index_name, doc_type='document',id=ID)['_source']['label']
    if label=='spam':
        ID_label='1'
    else:
        ID_label='0'
    
    body=es.get(index=index_name, doc_type='document',id=ID)['_source']['body']
    text=re.sub(" \d+", " ", body)
    text=text.replace('   ',' ')
    text=text.replace('  ',' ')
    
    texts=text.split()
    word_seq=dict()
    for text in texts:
        try:
            word_seq[str(text)]=word_dict[str(text)]
            try:
                term_freq[str(text)]+=1
            except:
                term_freq[str(text)]=1
        except:
            continue
    
    final=list()
    check_word=dict()
    
    for word in sorted(word_seq, key=word_seq.get):
        if word in check_word:
            continue
        else:
            check_word[word]=word
            final.append(str(word_dict[word]) + ":" + str(term_freq[word]))
            
    final_list=' '.join(final)
    sparse_matrix_test.write(ID_label+" "+final_list+"\n")
sparse_matrix_test.close()  
            



In [112]:
from liblinearutil import *


In [113]:
y, x = svm_read_problem('../sparse_matrix/sparse_matrix_train.txt')

In [115]:
y_test, x_test = svm_read_problem('../sparse_matrix_test.txt')

In [116]:
prob  = problem(y, x)
param = parameter('-s 0 -c 4 -B 1')

In [118]:
m = train(prob, param)
p_labels, p_acc, p_vals = predict(y_test, x_test, m,'-b 1')

Accuracy = 98.4753% (14855/15085) (classification)
